The data from this exercise comes from the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/Car+Evaluation For more details on the data set see the included documentation.

In [4]:
import pandas as pd
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
import aequitas.plot as ap

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, label_binarize, LabelBinarizer
from sklearn.metrics import f1_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
# I manually added the headers to the data set.
df = pd.read_csv("./car.csv")

# We'll modify the data to make it a binary problem of acceptable or unacceptable car.
df = df.where(df != 'good', 'acc')
df = df.where(df != 'vgood', 'acc')

y = df.pop('car')
X = df

df.head()

,buying,maint,doors,persons,lug_boot,safety
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=23)
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)

# Use this later to construct the DataFrame Aequitas requires.
df_aq = X_test.copy()

ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
X_train = ohe.fit_transform(X_train.values)
X_test = ohe.transform(X_test.values)

lb = LabelBinarizer()
y_train = label_binarize(y_train.values, classes=['unacc', 'acc']).ravel()
y_test = label_binarize(y_test.values, classes=['unacc', 'acc']).ravel()

X_train shape:  (1296, 6)
X_test shape:  (432, 6)


In [7]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

scores = lr.predict_proba(X_test)
pred = lr.predict(X_test)

f1 = f1_score(y_test, pred)
print(f"F1 score: {f1:.4f}")

F1 score: 0.8960


In [25]:
# Construct the dataframe that Aequitas will use.
# score_tresholded, correct_label, details
df_aq['score'] = pred
df_aq['label_value'] = y_test
df_aq.head(4)

,buying,maint,doors,persons,lug_boot,safety,score,label_value
1278,med,low,5more,4,small,low,0,0
477,high,vhigh,3,more,small,low,0,0
1688,low,low,4,4,med,high,1,1
1444,low,high,3,4,med,med,1,1


In [21]:
# Run Aequitas.
group = Group()
xtab, _ = group.get_crosstabs(df_aq)
xtab.head(10)

,model_id,score_threshold,k,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,pp,pn,ppr,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev
0,0,binary 0/1,131,buying,high,0.913043,0.900000,0.024096,0.300000,0.100000,0.086957,0.975904,0.700000,30,83,0.229008,0.265487,9,2,81,21,23,90,113,432,0.203540
1,0,binary 0/1,131,buying,low,0.972973,0.963636,0.018519,0.052632,0.036364,0.027027,0.981481,0.947368,38,54,0.290076,0.413043,2,1,53,36,37,55,92,432,0.402174
2,0,binary 0/1,131,buying,med,1.000000,0.946667,0.000000,0.088889,0.053333,0.000000,1.000000,0.911111,45,71,0.343511,0.387931,4,0,71,41,41,75,116,432,0.353448
3,0,binary 0/1,131,buying,vhigh,0.777778,0.956989,0.043011,0.222222,0.043011,0.222222,0.956989,0.777778,18,93,0.137405,0.162162,4,4,89,14,18,93,111,432,0.162162
4,0,binary 0/1,131,maint,high,0.896552,0.929412,0.036585,0.187500,0.070588,0.103448,0.963415,0.812500,32,82,0.244275,0.280702,6,3,79,26,29,85,114,432,0.254386
5,0,binary 0/1,131,maint,low,0.948718,0.916667,0.029412,0.139535,0.083333,0.051282,0.970588,0.860465,43,68,0.328244,0.387387,6,2,66,37,39,72,111,432,0.351351
6,0,binary 0/1,131,maint,med,0.947368,0.923077,0.032258,0.121951,0.076923,0.052632,0.967742,0.878049,41,62,0.312977,0.398058,5,2,60,36,38,65,103,432,0.368932
7,0,binary 0/1,131,maint,vhigh,1.000000,0.978022,0.000000,0.133333,0.021978,0.000000,1.000000,0.866667,15,89,0.114504,0.144231,2,0,89,13,13,91,104,432,0.125000
8,0,binary 0/1,131,doors,2,0.850000,0.938272,0.037975,0.227273,0.061728,0.150000,0.962025,0.772727,22,79,0.167939,0.217822,5,3,76,17,20,81,101,432,0.198020
9,0,binary 0/1,131,doors,3,0.921053,0.937500,0.038462,0.125000,0.062500,0.078947,0.961538,0.875000,40,78,0.305344,0.338983,5,3,75,35,38,80,118,432,0.322034


## Compute Bias
we calculate the bias vs a predefined group we manually set

In [38]:
bias = Bias()
bias_df = bias.get_disparity_predefined_groups(xtab,
                                               original_df=df_aq,
                                               ref_groups_dict={"safety": "high", "lug_boot": "med", "persons": "4", "doors": "4", "maint": "high", "buying": "med"},
                                               alpha=0.05,
                                               mask_significance=True
                                               )
bias_df.head(10)

get_disparity_predefined_group()


,model_id,score_threshold,k,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,pp,pn,ppr,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev,ppr_disparity,pprev_disparity,precision_disparity,fdr_disparity,for_disparity,fpr_disparity,fnr_disparity,tpr_disparity,tnr_disparity,npv_disparity,ppr_ref_group_value,pprev_ref_group_value,precision_ref_group_value,fdr_ref_group_value,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value
0,0,binary 0/1,131,buying,high,0.913043,0.900000,0.024096,0.300000,0.100000,0.086957,0.975904,0.700000,30,83,0.229008,0.265487,9,2,81,21,23,90,113,432,0.203540,0.666667,0.684366,0.768293,3.375000,10.000000,1.875000,10.000000,0.913043,0.950704,0.975904,med,med,med,med,med,med,med,med,med,med
1,0,binary 0/1,131,buying,low,0.972973,0.963636,0.018519,0.052632,0.036364,0.027027,0.981481,0.947368,38,54,0.290076,0.413043,2,1,53,36,37,55,92,432,0.402174,0.844444,1.064734,1.039795,0.592105,10.000000,0.681818,10.000000,0.972973,1.017926,0.981481,med,med,med,med,med,med,med,med,med,med
2,0,binary 0/1,131,buying,med,1.000000,0.946667,0.000000,0.088889,0.053333,0.000000,1.000000,0.911111,45,71,0.343511,0.387931,4,0,71,41,41,75,116,432,0.353448,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,NaN,1.000000,1.000000,1.000000,med,med,med,med,med,med,med,med,med,med
3,0,binary 0/1,131,buying,vhigh,0.777778,0.956989,0.043011,0.222222,0.043011,0.222222,0.956989,0.777778,18,93,0.137405,0.162162,4,4,89,14,18,93,111,432,0.162162,0.400000,0.418018,0.853659,2.500000,10.000000,0.806452,10.000000,0.777778,1.010904,0.956989,med,med,med,med,med,med,med,med,med,med
4,0,binary 0/1,131,maint,high,0.896552,0.929412,0.036585,0.187500,0.070588,0.103448,0.963415,0.812500,32,82,0.244275,0.280702,6,3,79,26,29,85,114,432,0.254386,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,high,high,high,high,high,high,high,high,high,high
5,0,binary 0/1,131,maint,low,0.948718,0.916667,0.029412,0.139535,0.083333,0.051282,0.970588,0.860465,43,68,0.328244,0.387387,6,2,66,37,39,72,111,432,0.351351,1.343750,1.380068,1.059034,0.744186,0.803922,1.180556,0.495726,1.058185,0.986287,1.007446,high,high,high,high,high,high,high,high,high,high
6,0,binary 0/1,131,maint,med,0.947368,0.923077,0.032258,0.121951,0.076923,0.052632,0.967742,0.878049,41,62,0.312977,0.398058,5,2,60,36,38,65,103,432,0.368932,1.281250,1.418083,1.080675,0.650407,0.881720,1.089744,0.508772,1.056680,0.993184,1.004492,high,high,high,high,high,high,high,high,high,high
7,0,binary 0/1,131,maint,vhigh,1.000000,0.978022,0.000000,0.133333,0.021978,0.000000,1.000000,0.866667,15,89,0.114504,0.144231,2,0,89,13,13,91,104,432,0.125000,0.468750,0.513822,1.066667,0.711111,0.000000,0.311355,0.000000,1.115385,1.052302,1.037975,high,high,high,high,high,high,high,high,high,high
8,0,binary 0/1,131,doors,2,0.850000,0.938272,0.037975,0.227273,0.061728,0.150000,0.962025,0.772727,22,79,0.167939,0.217822,5,3,76,17,20,81,101,432,0.198020,0.647059,0.704718,0.905956,1.545455,10.000000,1.000000,10.000000,0.850000,1.000000,0.962025,4,4,4,4,4,4,4,4,4,4
9,0,binary 0/1,131,doors,3,0.921053,0.937500,0.038462,0.125000,0.062500,0.078947,0.961538,0.875000,40,78,0.305344,0.338983,5,3,75,35,38,80,118,432,0.322034,1.176471,1.096710,1.025862,0.850000,10.000000,1.012500,10.000000,0.921053,0.999178,0.961538,4,4,4,4,4,4,4,4,4,4


Another common option:

In [32]:
bias_df = bias.get_disparity_major_group(xtab,
                                         original_df=df_aq,
                                         alpha=0.05,
                                         mask_significance=True
                                         )
bias_df.head(10)

get_disparity_major_group()


,model_id,score_threshold,k,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,pp,pn,ppr,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev,ppr_disparity,pprev_disparity,precision_disparity,fdr_disparity,for_disparity,fpr_disparity,fnr_disparity,tpr_disparity,tnr_disparity,npv_disparity,ppr_ref_group_value,pprev_ref_group_value,precision_ref_group_value,fdr_ref_group_value,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value
0,0,binary 0/1,131,buying,high,0.913043,0.900000,0.024096,0.300000,0.100000,0.086957,0.975904,0.700000,30,83,0.229008,0.265487,9,2,81,21,23,90,113,432,0.203540,0.666667,0.684366,0.768293,3.375000,10.000000,1.875000,10.000000,0.913043,0.950704,0.975904,med,med,med,med,med,med,med,med,med,med
1,0,binary 0/1,131,buying,low,0.972973,0.963636,0.018519,0.052632,0.036364,0.027027,0.981481,0.947368,38,54,0.290076,0.413043,2,1,53,36,37,55,92,432,0.402174,0.844444,1.064734,1.039795,0.592105,10.000000,0.681818,10.000000,0.972973,1.017926,0.981481,med,med,med,med,med,med,med,med,med,med
2,0,binary 0/1,131,buying,med,1.000000,0.946667,0.000000,0.088889,0.053333,0.000000,1.000000,0.911111,45,71,0.343511,0.387931,4,0,71,41,41,75,116,432,0.353448,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,NaN,1.000000,1.000000,1.000000,med,med,med,med,med,med,med,med,med,med
3,0,binary 0/1,131,buying,vhigh,0.777778,0.956989,0.043011,0.222222,0.043011,0.222222,0.956989,0.777778,18,93,0.137405,0.162162,4,4,89,14,18,93,111,432,0.162162,0.400000,0.418018,0.853659,2.500000,10.000000,0.806452,10.000000,0.777778,1.010904,0.956989,med,med,med,med,med,med,med,med,med,med
4,0,binary 0/1,131,maint,high,0.896552,0.929412,0.036585,0.187500,0.070588,0.103448,0.963415,0.812500,32,82,0.244275,0.280702,6,3,79,26,29,85,114,432,0.254386,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,high,high,high,high,high,high,high,high,high,high
5,0,binary 0/1,131,maint,low,0.948718,0.916667,0.029412,0.139535,0.083333,0.051282,0.970588,0.860465,43,68,0.328244,0.387387,6,2,66,37,39,72,111,432,0.351351,1.343750,1.380068,1.059034,0.744186,0.803922,1.180556,0.495726,1.058185,0.986287,1.007446,high,high,high,high,high,high,high,high,high,high
6,0,binary 0/1,131,maint,med,0.947368,0.923077,0.032258,0.121951,0.076923,0.052632,0.967742,0.878049,41,62,0.312977,0.398058,5,2,60,36,38,65,103,432,0.368932,1.281250,1.418083,1.080675,0.650407,0.881720,1.089744,0.508772,1.056680,0.993184,1.004492,high,high,high,high,high,high,high,high,high,high
7,0,binary 0/1,131,maint,vhigh,1.000000,0.978022,0.000000,0.133333,0.021978,0.000000,1.000000,0.866667,15,89,0.114504,0.144231,2,0,89,13,13,91,104,432,0.125000,0.468750,0.513822,1.066667,0.711111,0.000000,0.311355,0.000000,1.115385,1.052302,1.037975,high,high,high,high,high,high,high,high,high,high
8,0,binary 0/1,131,doors,2,0.850000,0.938272,0.037975,0.227273,0.061728,0.150000,0.962025,0.772727,22,79,0.167939,0.217822,5,3,76,17,20,81,101,432,0.198020,0.550000,0.642574,0.883117,1.818182,0.987342,0.987654,1.900000,0.922857,1.000823,1.000506,3,3,3,3,3,3,3,3,3,3
9,0,binary 0/1,131,doors,3,0.921053,0.937500,0.038462,0.125000,0.062500,0.078947,0.961538,0.875000,40,78,0.305344,0.338983,5,3,75,35,38,80,118,432,0.322034,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3,3,3,3,3,3,3,3,3,3


In [39]:
fairness = Fairness()
fairness_df = fairness.get_group_value_fairness(bias_df)
fairness_df.head(10)

,model_id,score_threshold,k,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,pp,pn,ppr,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev,ppr_disparity,pprev_disparity,precision_disparity,fdr_disparity,for_disparity,fpr_disparity,fnr_disparity,tpr_disparity,tnr_disparity,npv_disparity,ppr_ref_group_value,pprev_ref_group_value,precision_ref_group_value,fdr_ref_group_value,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value,Statistical Parity,Impact Parity,FDR Parity,FPR Parity,FOR Parity,FNR Parity,TPR Parity,TNR Parity,NPV Parity,Precision Parity,TypeI Parity,TypeII Parity,Equalized Odds,Unsupervised Fairness,Supervised Fairness
0,0,binary 0/1,131,buying,high,0.913043,0.900000,0.024096,0.300000,0.100000,0.086957,0.975904,0.700000,30,83,0.229008,0.265487,9,2,81,21,23,90,113,432,0.203540,0.666667,0.684366,0.768293,3.375000,10.000000,1.875000,10.000000,0.913043,0.950704,0.975904,med,med,med,med,med,med,med,med,med,med,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False
1,0,binary 0/1,131,buying,low,0.972973,0.963636,0.018519,0.052632,0.036364,0.027027,0.981481,0.947368,38,54,0.290076,0.413043,2,1,53,36,37,55,92,432,0.402174,0.844444,1.064734,1.039795,0.592105,10.000000,0.681818,10.000000,0.972973,1.017926,0.981481,med,med,med,med,med,med,med,med,med,med,True,True,False,False,False,False,True,True,True,True,False,False,False,True,False
2,0,binary 0/1,131,buying,med,1.000000,0.946667,0.000000,0.088889,0.053333,0.000000,1.000000,0.911111,45,71,0.343511,0.387931,4,0,71,41,41,75,116,432,0.353448,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,NaN,1.000000,1.000000,1.000000,med,med,med,med,med,med,med,med,med,med,True,True,True,True,NaN,NaN,True,True,True,True,True,NaN,True,True,False
3,0,binary 0/1,131,buying,vhigh,0.777778,0.956989,0.043011,0.222222,0.043011,0.222222,0.956989,0.777778,18,93,0.137405,0.162162,4,4,89,14,18,93,111,432,0.162162,0.400000,0.418018,0.853659,2.500000,10.000000,0.806452,10.000000,0.777778,1.010904,0.956989,med,med,med,med,med,med,med,med,med,med,False,False,False,True,False,False,False,True,True,True,False,False,False,False,False
4,0,binary 0/1,131,maint,high,0.896552,0.929412,0.036585,0.187500,0.070588,0.103448,0.963415,0.812500,32,82,0.244275,0.280702,6,3,79,26,29,85,114,432,0.254386,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,high,high,high,high,high,high,high,high,high,high,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
5,0,binary 0/1,131,maint,low,0.948718,0.916667,0.029412,0.139535,0.083333,0.051282,0.970588,0.860465,43,68,0.328244,0.387387,6,2,66,37,39,72,111,432,0.351351,1.343750,1.380068,1.059034,0.744186,0.803922,1.180556,0.495726,1.058185,0.986287,1.007446,high,high,high,high,high,high,high,high,high,high,False,False,False,True,True,False,True,True,True,True,False,False,True,False,False
6,0,binary 0/1,131,maint,med,0.947368,0.923077,0.032258,0.121951,0.076923,0.052632,0.967742,0.878049,41,62,0.312977,0.398058,5,2,60,36,38,65,103,432,0.368932,1.281250,1.418083,1.080675,0.650407,0.881720,1.089744,0.508772,1.056680,0.993184,1.004492,high,high,high,high,high,high,high,high,high,high,False,False,False,True,True,False,True,True,True,True,False,False,True,False,False
7,0,binary 0/1,131,maint,vhigh,1.000000,0.978022,0.000000,0.133333,0.021978,0.000000,1.000000,0.866667,15,89,0.114504,0.144231,2,0,89,13,13,91,104,432,0.125000,0.468750,0.513822,1.066667,0.711111,0.000000,0.311355,0.000000,1.115385,1.052302,1.037975,high,high,high,high,high,high,high,high,high,high,False,False,False,False,False,False,True,True,True,True,False,False,False,False,False
8,0,binary 0/1,131,doors,2,0.850000,0.938272,0.037975,0.227273,0.061728,0.150000,0.962025,0.772727,22,79,0.167939,0.217822,5,3,76,17,20,81,101,432,0.198020,0.647059,0.704718,0.905956,1.545455,10.000000,1.000000,10.000000,0.850000,1.000000,0

In [40]:
overall_fairness = fairness.get_overall_fairness(fairness_df)
print(overall_fairness)

{'Unsupervised Fairness': False, 'Supervised Fairness': False, 'Overall Fairness': False}


In [42]:
metrics = ['fpr', 'fnr', 'for']
disparity_tolerance = 1.30

ap.summary(bias_df, metrics, fairness_threshold=disparity_tolerance)

ZeroDivisionError: float division by zero